In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from datetime import datetime
import time
import warnings
warnings.filterwarnings('ignore')

In [2]:
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans
from sklearn.model_selection import KFold, cross_val_score
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone


In [3]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor


In [4]:
train = pd.read_csv('./final_train.csv',index_col=0)
test = pd.read_csv('./final_test.csv',index_col=0)

In [5]:
train.columns

Index(['area', 'houseFloor', 'totalFloor', 'houseToward', 'houseDecoration',
       'plate', 'buildYear', 'saleSecHouseNum', 'subwayStationNum',
       'busStationNum',
       ...
       'communityNameW1', 'communityNameW2', 'communityNameW3',
       'communityNameW4', 'communityNameW5', 'communityNameW6',
       'communityNameW7', 'communityNameW8', 'communityNameW9', 'tradeMoney'],
      dtype='object', length=112)

In [6]:
from sklearn.metrics import r2_score
def online_score(pred):
    print("预测结果最大值：{},预测结果最小值：{}".format(pred.max(),pred.min()))
    # a榜测分
    conmbine1 = pd.read_csv("./sub_b_919.csv",engine = "python",header=None)
    score1 = r2_score(list(map(lambda x:int(x),pred)), conmbine1.values)
    print("对比919分数:{}".format(score1))

In [7]:
categorical_feats = [ 'houseFloor', 'houseToward', 'houseDecoration', 'plate']
le = LabelEncoder()
for i in categorical_feats:
    train[i] = le.fit_transform(train[i])
    test[i] = le.fit_transform(test[i])

In [8]:
target = train.pop('tradeMoney')

In [24]:
lightgbm = LGBMRegressor(num_leaves=100,
                         n_estimators=3000,
                        min_data_in_leaf=20,
                        min_child_samples=20,
                        objective= 'regression',
                        learning_rate= 0.01,
                        boosting= "gbdt",
                        feature_fraction= 0.8,
                        bagging_freq =1,
                        bagging_fraction=0.85,
                        bagging_seed=23,
                        metric='rmse',
                        lambda_l1=0.277,
                        nthread=-1)
start = time.time()
lightgbm.fit(train,target)
pred1 = lightgbm.predict(train)
pred2 = lightgbm.predict(test)
end = time.time()
print('train loss:{}'.format(r2_score(pred1,target)))
# print('test loss:{}'.format(online_score(pred2)))
print('耗时{}'.format(end-start))

train loss:0.9730419575136059
预测结果最大值：15740.111745199163,预测结果最小值：1191.3086760964484
对比919分数:0.9829000853720631
test loss:None
耗时161.16584372520447


In [22]:
import catboost as cb
model_cb = cb.CatBoostRegressor(iterations=1000, depth=7, learning_rate=0.9,eval_metric = "R2",leaf_estimation_method = "Gradient",
                            l2_leaf_reg=3)
y_train = target
# 采用平滑后的y_train进行模型训练
model_cb.fit(train, y_train)
# 对训练集预测
pred_log1p = model_cb.predict(train)
# 模型评分
score = r2_score(y_train,pred_log1p)
print('train score:{}'.format(score))
# 对测试集预测
CB_pred = model_cb.predict(test)
online_score(CB_pred)
# print('test score:{}'.format())


0:	learn: 0.6300294	total: 578ms	remaining: 9m 37s
1:	learn: 0.7655100	total: 1.17s	remaining: 9m 41s
2:	learn: 0.8009522	total: 1.75s	remaining: 9m 43s
3:	learn: 0.8219672	total: 2.27s	remaining: 9m 24s
4:	learn: 0.8369556	total: 2.81s	remaining: 9m 18s
5:	learn: 0.8463540	total: 3.39s	remaining: 9m 21s
6:	learn: 0.8522063	total: 3.88s	remaining: 9m 10s
7:	learn: 0.8549245	total: 4.38s	remaining: 9m 2s
8:	learn: 0.8600149	total: 4.89s	remaining: 8m 58s
9:	learn: 0.8661427	total: 5.4s	remaining: 8m 55s
10:	learn: 0.8702131	total: 5.9s	remaining: 8m 50s
11:	learn: 0.8702954	total: 6.05s	remaining: 8m 18s
12:	learn: 0.8727731	total: 6.54s	remaining: 8m 16s
13:	learn: 0.8729828	total: 6.82s	remaining: 8m
14:	learn: 0.8730220	total: 7.08s	remaining: 7m 45s
15:	learn: 0.8747950	total: 7.56s	remaining: 7m 44s
16:	learn: 0.8779019	total: 8.06s	remaining: 7m 45s
17:	learn: 0.8819288	total: 8.65s	remaining: 7m 52s
18:	learn: 0.8839938	total: 9.27s	remaining: 7m 58s
19:	learn: 0.8875877	total: 9

158:	learn: 0.9227134	total: 1m 23s	remaining: 7m 20s
159:	learn: 0.9227314	total: 1m 23s	remaining: 7m 20s
160:	learn: 0.9227500	total: 1m 24s	remaining: 7m 20s
161:	learn: 0.9227648	total: 1m 25s	remaining: 7m 20s
162:	learn: 0.9227668	total: 1m 25s	remaining: 7m 20s
163:	learn: 0.9229692	total: 1m 26s	remaining: 7m 20s
164:	learn: 0.9229892	total: 1m 27s	remaining: 7m 20s
165:	learn: 0.9230050	total: 1m 27s	remaining: 7m 20s
166:	learn: 0.9230050	total: 1m 27s	remaining: 7m 17s
167:	learn: 0.9230972	total: 1m 28s	remaining: 7m 17s
168:	learn: 0.9232971	total: 1m 28s	remaining: 7m 16s
169:	learn: 0.9233347	total: 1m 29s	remaining: 7m 16s
170:	learn: 0.9233407	total: 1m 29s	remaining: 7m 15s
171:	learn: 0.9233450	total: 1m 30s	remaining: 7m 15s
172:	learn: 0.9233621	total: 1m 31s	remaining: 7m 16s
173:	learn: 0.9234040	total: 1m 31s	remaining: 7m 16s
174:	learn: 0.9234078	total: 1m 32s	remaining: 7m 15s
175:	learn: 0.9234079	total: 1m 32s	remaining: 7m 14s
176:	learn: 0.9234291	total:

312:	learn: 0.9259456	total: 2m 43s	remaining: 5m 58s
313:	learn: 0.9259518	total: 2m 43s	remaining: 5m 57s
314:	learn: 0.9259780	total: 2m 44s	remaining: 5m 56s
315:	learn: 0.9259794	total: 2m 44s	remaining: 5m 56s
316:	learn: 0.9260154	total: 2m 45s	remaining: 5m 55s
317:	learn: 0.9260163	total: 2m 45s	remaining: 5m 55s
318:	learn: 0.9260943	total: 2m 46s	remaining: 5m 54s
319:	learn: 0.9261313	total: 2m 46s	remaining: 5m 53s
320:	learn: 0.9262168	total: 2m 46s	remaining: 5m 53s
321:	learn: 0.9263271	total: 2m 47s	remaining: 5m 52s
322:	learn: 0.9271896	total: 2m 47s	remaining: 5m 52s
323:	learn: 0.9271975	total: 2m 48s	remaining: 5m 51s
324:	learn: 0.9272178	total: 2m 48s	remaining: 5m 50s
325:	learn: 0.9272385	total: 2m 49s	remaining: 5m 50s
326:	learn: 0.9272663	total: 2m 49s	remaining: 5m 49s
327:	learn: 0.9272795	total: 2m 50s	remaining: 5m 49s
328:	learn: 0.9273410	total: 2m 50s	remaining: 5m 48s
329:	learn: 0.9273950	total: 2m 51s	remaining: 5m 47s
330:	learn: 0.9274123	total:

465:	learn: 0.9305325	total: 3m 55s	remaining: 4m 29s
466:	learn: 0.9305403	total: 3m 55s	remaining: 4m 29s
467:	learn: 0.9305710	total: 3m 56s	remaining: 4m 28s
468:	learn: 0.9305917	total: 3m 56s	remaining: 4m 28s
469:	learn: 0.9305963	total: 3m 57s	remaining: 4m 27s
470:	learn: 0.9305974	total: 3m 57s	remaining: 4m 27s
471:	learn: 0.9305976	total: 3m 58s	remaining: 4m 26s
472:	learn: 0.9306068	total: 3m 59s	remaining: 4m 26s
473:	learn: 0.9306082	total: 3m 59s	remaining: 4m 25s
474:	learn: 0.9306082	total: 4m	remaining: 4m 25s
475:	learn: 0.9306082	total: 4m	remaining: 4m 24s
476:	learn: 0.9307790	total: 4m 1s	remaining: 4m 24s
477:	learn: 0.9307807	total: 4m 1s	remaining: 4m 23s
478:	learn: 0.9307816	total: 4m 2s	remaining: 4m 23s
479:	learn: 0.9307867	total: 4m 2s	remaining: 4m 22s
480:	learn: 0.9307960	total: 4m 3s	remaining: 4m 22s
481:	learn: 0.9307996	total: 4m 3s	remaining: 4m 22s
482:	learn: 0.9308016	total: 4m 4s	remaining: 4m 21s
483:	learn: 0.9308126	total: 4m 4s	remainin

618:	learn: 0.9340398	total: 5m 35s	remaining: 3m 26s
619:	learn: 0.9340500	total: 5m 35s	remaining: 3m 25s
620:	learn: 0.9342130	total: 5m 36s	remaining: 3m 25s
621:	learn: 0.9342132	total: 5m 36s	remaining: 3m 24s
622:	learn: 0.9342186	total: 5m 37s	remaining: 3m 24s
623:	learn: 0.9342293	total: 5m 38s	remaining: 3m 23s
624:	learn: 0.9343484	total: 5m 38s	remaining: 3m 23s
625:	learn: 0.9343676	total: 5m 39s	remaining: 3m 22s
626:	learn: 0.9343747	total: 5m 39s	remaining: 3m 22s
627:	learn: 0.9343844	total: 5m 40s	remaining: 3m 21s
628:	learn: 0.9344090	total: 5m 40s	remaining: 3m 21s
629:	learn: 0.9344092	total: 5m 41s	remaining: 3m 20s
630:	learn: 0.9344096	total: 5m 41s	remaining: 3m 19s
631:	learn: 0.9344099	total: 5m 42s	remaining: 3m 19s
632:	learn: 0.9345615	total: 5m 42s	remaining: 3m 18s
633:	learn: 0.9345638	total: 5m 43s	remaining: 3m 18s
634:	learn: 0.9345675	total: 5m 43s	remaining: 3m 17s
635:	learn: 0.9345686	total: 5m 44s	remaining: 3m 17s
636:	learn: 0.9345949	total:

772:	learn: 0.9356431	total: 7m 6s	remaining: 2m 5s
773:	learn: 0.9356436	total: 7m 7s	remaining: 2m 4s
774:	learn: 0.9356514	total: 7m 7s	remaining: 2m 4s
775:	learn: 0.9356731	total: 7m 8s	remaining: 2m 3s
776:	learn: 0.9356731	total: 7m 8s	remaining: 2m 3s
777:	learn: 0.9356858	total: 7m 9s	remaining: 2m 2s
778:	learn: 0.9356895	total: 7m 10s	remaining: 2m 2s
779:	learn: 0.9356895	total: 7m 10s	remaining: 2m 1s
780:	learn: 0.9357044	total: 7m 10s	remaining: 2m
781:	learn: 0.9357167	total: 7m 11s	remaining: 2m
782:	learn: 0.9357171	total: 7m 12s	remaining: 1m 59s
783:	learn: 0.9357239	total: 7m 12s	remaining: 1m 59s
784:	learn: 0.9357241	total: 7m 13s	remaining: 1m 58s
785:	learn: 0.9357249	total: 7m 14s	remaining: 1m 58s
786:	learn: 0.9357259	total: 7m 15s	remaining: 1m 57s
787:	learn: 0.9357430	total: 7m 16s	remaining: 1m 57s
788:	learn: 0.9357433	total: 7m 16s	remaining: 1m 56s
789:	learn: 0.9357438	total: 7m 17s	remaining: 1m 56s
790:	learn: 0.9357571	total: 7m 18s	remaining: 1m 

926:	learn: 0.9368118	total: 8m 37s	remaining: 40.7s
927:	learn: 0.9368141	total: 8m 37s	remaining: 40.2s
928:	learn: 0.9368152	total: 8m 38s	remaining: 39.6s
929:	learn: 0.9368153	total: 8m 38s	remaining: 39s
930:	learn: 0.9368153	total: 8m 38s	remaining: 38.4s
931:	learn: 0.9368222	total: 8m 39s	remaining: 37.9s
932:	learn: 0.9368239	total: 8m 39s	remaining: 37.3s
933:	learn: 0.9368294	total: 8m 40s	remaining: 36.8s
934:	learn: 0.9368379	total: 8m 40s	remaining: 36.2s
935:	learn: 0.9368384	total: 8m 41s	remaining: 35.6s
936:	learn: 0.9368385	total: 8m 41s	remaining: 35.1s
937:	learn: 0.9368400	total: 8m 41s	remaining: 34.5s
938:	learn: 0.9368419	total: 8m 42s	remaining: 33.9s
939:	learn: 0.9368465	total: 8m 42s	remaining: 33.4s
940:	learn: 0.9368470	total: 8m 43s	remaining: 32.8s
941:	learn: 0.9368503	total: 8m 43s	remaining: 32.3s
942:	learn: 0.9368522	total: 8m 44s	remaining: 31.7s
943:	learn: 0.9368522	total: 8m 44s	remaining: 31.1s
944:	learn: 0.9368522	total: 8m 44s	remaining: 3

In [28]:
model_xgb = XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=6, 
                             min_child_weight=1.7817, n_estimators=5000,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1,)
# 进行模型训练
model_xgb.fit(train, target)
# 对训练集预测
pred_log1p = model_xgb.predict(train)
# 模型评分
score = r2_score(target,pred_log1p)
print('train score:{}'.format(score))
# 对测试集预测
xgb_pred = model_xgb.predict(test)
# 保存文件
print('test score:{}'.format(online_score(xgb_pred)))


train score:0.9850817159626464
预测结果最大值：15633.9453125,预测结果最小值：1097.5894775390625
对比919分数:0.9717013091617863
test score:None


In [9]:
from mlxtend.regressor import StackingCVRegressor
import catboost as cb

lightgbm = LGBMRegressor(num_leaves=100,
                         n_estimators=3000,
                        min_data_in_leaf=20,
                        min_child_samples=20,
                        objective= 'regression',
                        learning_rate= 0.01,
                        boosting= "gbdt",
                        feature_fraction= 0.8,
                        bagging_freq =1,
                        bagging_fraction=0.85,
                        bagging_seed=23,
                        metric='rmse',
                        lambda_l1=0.277,
                        nthread=-1)
model_xgb = XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=6, 
                             min_child_weight=1.7817, n_estimators=5000,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1,)
model_cb = cb.CatBoostRegressor(iterations=1000, depth=6, learning_rate=0.9,eval_metric = "R2",leaf_estimation_method = "Newton",
                            l2_leaf_reg=3)


In [ ]:
stack_gen = StackingCVRegressor(regressors=( lightgbm, model_xgb),
                                meta_regressor=model_cb,
                                use_features_in_secondary=True)
features = train.columns
y_train = target
# 采用平滑后的y_train进行模型训练
# train1 = np.array(train.values)
# test1 = np.array(test.values)
# log1p_y_train = np.array(log1p_y_train.values)
stregr = stack_gen.fit(train, y_train)
# 对训练集预测
pred_log1p = stregr.predict(train)
# 模型评分
score = r2_score(y_train,pred_log1p)
print(score)
# 对测试集预测
stregr_pred = stregr.predict(test)
# 将测试集结果去掉平滑
# stregr_pred = np.expm1(stregr_pred)
print(stregr_pred.max(),stregr_pred.min())
print('test score:{}'.format(online_score(gbr_pred)))


In [34]:
gbr = GradientBoostingRegressor(n_estimators=2000, learning_rate=0.05, max_depth=10, max_features='sqrt', min_samples_leaf=15, min_samples_split=10, loss='huber', random_state =2019)
gbr.fit(train, target)
# 对训练集预测
pred_gbr = gbr.predict(train)
# 模型评分
score = r2_score(target,pred_gbr)
print('train score:{}'.format(score))
# 对测试集预测
gbr_pred = gbr.predict(test)
# 保存文件
print('test score:{}'.format(online_score(gbr_pred)))


train score:0.9794207912170773
预测结果最大值：16332.072727790877,预测结果最小值：989.4725261432108
对比919分数:0.9728042160980667
test score:None


In [36]:
pred_all = xgb_pred + CB_pred +pred2
online_score(pred_all/3)
#简单平均后的结果

预测结果最大值：15516.608174790681,预测结果最小值：1154.7257543812184
对比919分数:0.9820204867242706


# 生成提交数据

In [11]:
class stacking(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self,mod,meta_model):
        self.mod = mod
        self.meta_model = meta_model
        self.kf = KFold(n_splits=3, random_state=42, shuffle=True)
        
    def fit(self,X,y):
        self.saved_model = [list() for i in self.mod]
        oof_train = np.zeros((X.shape[0], len(self.mod)))
        
        for i,model in enumerate(self.mod):
            for train_index, val_index in self.kf.split(X,y):
                renew_model = clone(model)
                renew_model.fit(X[train_index], y[train_index])
                self.saved_model[i].append(renew_model)
                oof_train[val_index,i] = renew_model.predict(X[val_index])
        
        self.meta_model.fit(oof_train,y)
        return self
    
    def predict(self,X):
        whole_test = np.column_stack([np.column_stack(model.predict(X) for model in single_model).mean(axis=1) 
                                      for single_model in self.saved_model]) 
        return self.meta_model.predict(whole_test)
    
    def get_oof(self,X,y,test_X):
        oof = np.zeros((X.shape[0],len(self.mod)))
        test_single = np.zeros((test_X.shape[0],5))
        test_mean = np.zeros((test_X.shape[0],len(self.mod)))
        for i,model in enumerate(self.mod):
            for j, (train_index,val_index) in enumerate(self.kf.split(X,y)):
                clone_model = clone(model)
                clone_model.fit(X[train_index],y[train_index])
                oof[val_index,i] = clone_model.predict(X[val_index])
                test_single[:,j] = clone_model.predict(test_X)
            test_mean[:,i] = test_single.mean(axis=1)
        return oof, test_mean

In [12]:
def cv_rmse(model,x, y):
    rmse= np.sqrt(-cross_val_score(model, x, y, scoring="neg_mean_squared_error", cv = 3))
    return(rmse)

In [13]:
stack_model = stacking(mod=[lightgbm, model_xgb,model_cb],meta_model=LinearRegression())

In [14]:
# score = cv_rmse(stack_model,train.values,target.values)
# print(score.mean())

In [15]:
stack_model.fit(train.values,target.values)

0:	learn: 0.4445084	total: 272ms	remaining: 4m 32s
1:	learn: 0.7161583	total: 464ms	remaining: 3m 51s
2:	learn: 0.7737729	total: 650ms	remaining: 3m 36s
3:	learn: 0.8038426	total: 846ms	remaining: 3m 30s
4:	learn: 0.8175815	total: 1.02s	remaining: 3m 22s
5:	learn: 0.8318331	total: 1.2s	remaining: 3m 19s
6:	learn: 0.8391050	total: 1.41s	remaining: 3m 20s
7:	learn: 0.8451292	total: 1.59s	remaining: 3m 17s
8:	learn: 0.8491961	total: 1.77s	remaining: 3m 14s
9:	learn: 0.8562607	total: 1.94s	remaining: 3m 12s
10:	learn: 0.8623841	total: 2.11s	remaining: 3m 9s
11:	learn: 0.8651007	total: 2.3s	remaining: 3m 9s
12:	learn: 0.8675190	total: 2.47s	remaining: 3m 7s
13:	learn: 0.8697626	total: 2.63s	remaining: 3m 5s
14:	learn: 0.8698548	total: 2.73s	remaining: 2m 59s
15:	learn: 0.8716535	total: 2.9s	remaining: 2m 58s
16:	learn: 0.8717792	total: 3.04s	remaining: 2m 56s
17:	learn: 0.8740446	total: 3.21s	remaining: 2m 55s
18:	learn: 0.8750753	total: 3.36s	remaining: 2m 53s
19:	learn: 0.8760946	total: 3

158:	learn: 0.9216340	total: 28.8s	remaining: 2m 32s
159:	learn: 0.9220090	total: 29s	remaining: 2m 32s
160:	learn: 0.9221424	total: 29.2s	remaining: 2m 32s
161:	learn: 0.9222290	total: 29.4s	remaining: 2m 31s
162:	learn: 0.9222671	total: 29.5s	remaining: 2m 31s
163:	learn: 0.9222793	total: 29.7s	remaining: 2m 31s
164:	learn: 0.9224057	total: 29.9s	remaining: 2m 31s
165:	learn: 0.9224249	total: 30.1s	remaining: 2m 31s
166:	learn: 0.9224792	total: 30.3s	remaining: 2m 31s
167:	learn: 0.9225286	total: 30.5s	remaining: 2m 31s
168:	learn: 0.9225317	total: 30.7s	remaining: 2m 30s
169:	learn: 0.9227670	total: 30.8s	remaining: 2m 30s
170:	learn: 0.9227816	total: 31s	remaining: 2m 30s
171:	learn: 0.9228212	total: 31.1s	remaining: 2m 29s
172:	learn: 0.9228606	total: 31.3s	remaining: 2m 29s
173:	learn: 0.9228815	total: 31.4s	remaining: 2m 29s
174:	learn: 0.9230805	total: 31.6s	remaining: 2m 29s
175:	learn: 0.9231120	total: 31.8s	remaining: 2m 28s
176:	learn: 0.9231137	total: 31.9s	remaining: 2m 2

315:	learn: 0.9298793	total: 55.4s	remaining: 1m 59s
316:	learn: 0.9298799	total: 55.6s	remaining: 1m 59s
317:	learn: 0.9298862	total: 55.8s	remaining: 1m 59s
318:	learn: 0.9300145	total: 56s	remaining: 1m 59s
319:	learn: 0.9300148	total: 56.2s	remaining: 1m 59s
320:	learn: 0.9300198	total: 56.3s	remaining: 1m 59s
321:	learn: 0.9300376	total: 56.5s	remaining: 1m 58s
322:	learn: 0.9300448	total: 56.7s	remaining: 1m 58s
323:	learn: 0.9300869	total: 56.9s	remaining: 1m 58s
324:	learn: 0.9300884	total: 57s	remaining: 1m 58s
325:	learn: 0.9300937	total: 57.2s	remaining: 1m 58s
326:	learn: 0.9300970	total: 57.4s	remaining: 1m 58s
327:	learn: 0.9301053	total: 57.6s	remaining: 1m 57s
328:	learn: 0.9301089	total: 57.7s	remaining: 1m 57s
329:	learn: 0.9301211	total: 57.9s	remaining: 1m 57s
330:	learn: 0.9301343	total: 58.1s	remaining: 1m 57s
331:	learn: 0.9301358	total: 58.3s	remaining: 1m 57s
332:	learn: 0.9301582	total: 58.5s	remaining: 1m 57s
333:	learn: 0.9302252	total: 58.7s	remaining: 1m 5

470:	learn: 0.9335406	total: 1m 24s	remaining: 1m 34s
471:	learn: 0.9335705	total: 1m 24s	remaining: 1m 34s
472:	learn: 0.9335710	total: 1m 24s	remaining: 1m 34s
473:	learn: 0.9336082	total: 1m 24s	remaining: 1m 34s
474:	learn: 0.9336091	total: 1m 25s	remaining: 1m 34s
475:	learn: 0.9336114	total: 1m 25s	remaining: 1m 34s
476:	learn: 0.9336117	total: 1m 25s	remaining: 1m 33s
477:	learn: 0.9336118	total: 1m 25s	remaining: 1m 33s
478:	learn: 0.9336325	total: 1m 26s	remaining: 1m 33s
479:	learn: 0.9336330	total: 1m 26s	remaining: 1m 33s
480:	learn: 0.9336416	total: 1m 26s	remaining: 1m 33s
481:	learn: 0.9336439	total: 1m 26s	remaining: 1m 33s
482:	learn: 0.9337893	total: 1m 26s	remaining: 1m 32s
483:	learn: 0.9337907	total: 1m 27s	remaining: 1m 32s
484:	learn: 0.9337909	total: 1m 27s	remaining: 1m 32s
485:	learn: 0.9338116	total: 1m 27s	remaining: 1m 32s
486:	learn: 0.9338119	total: 1m 27s	remaining: 1m 32s
487:	learn: 0.9338128	total: 1m 27s	remaining: 1m 32s
488:	learn: 0.9338170	total:

622:	learn: 0.9362519	total: 1m 54s	remaining: 1m 9s
623:	learn: 0.9362538	total: 1m 55s	remaining: 1m 9s
624:	learn: 0.9362584	total: 1m 55s	remaining: 1m 9s
625:	learn: 0.9362679	total: 1m 55s	remaining: 1m 8s
626:	learn: 0.9362897	total: 1m 55s	remaining: 1m 8s
627:	learn: 0.9362900	total: 1m 55s	remaining: 1m 8s
628:	learn: 0.9362901	total: 1m 55s	remaining: 1m 8s
629:	learn: 0.9362902	total: 1m 56s	remaining: 1m 8s
630:	learn: 0.9362970	total: 1m 56s	remaining: 1m 8s
631:	learn: 0.9363008	total: 1m 56s	remaining: 1m 7s
632:	learn: 0.9363025	total: 1m 56s	remaining: 1m 7s
633:	learn: 0.9363044	total: 1m 56s	remaining: 1m 7s
634:	learn: 0.9363119	total: 1m 57s	remaining: 1m 7s
635:	learn: 0.9363154	total: 1m 57s	remaining: 1m 7s
636:	learn: 0.9363155	total: 1m 57s	remaining: 1m 6s
637:	learn: 0.9363190	total: 1m 57s	remaining: 1m 6s
638:	learn: 0.9363305	total: 1m 57s	remaining: 1m 6s
639:	learn: 0.9363326	total: 1m 57s	remaining: 1m 6s
640:	learn: 0.9363330	total: 1m 58s	remaining:

...

501:	learn: 0.9288151	total: 2m	remaining: 1m 59s
502:	learn: 0.9288210	total: 2m	remaining: 1m 59s
503:	learn: 0.9289045	total: 2m	remaining: 1m 59s
504:	learn: 0.9289120	total: 2m 1s	remaining: 1m 58s
505:	learn: 0.9289360	total: 2m 1s	remaining: 1m 58s
506:	learn: 0.9289422	total: 2m 1s	remaining: 1m 58s
507:	learn: 0.9289815	total: 2m 1s	remaining: 1m 57s
508:	learn: 0.9290347	total: 2m 2s	remaining: 1m 57s
509:	learn: 0.9290353	total: 2m 2s	remaining: 1m 57s
510:	learn: 0.9290436	total: 2m 2s	remaining: 1m 57s
511:	learn: 0.9290715	total: 2m 2s	remaining: 1m 56s
512:	learn: 0.9290890	total: 2m 2s	remaining: 1m 56s
513:	learn: 0.9290905	total: 2m 3s	remaining: 1m 56s
514:	learn: 0.9290913	total: 2m 3s	remaining: 1m 56s
515:	learn: 0.9291082	total: 2m 3s	remaining: 1m 56s
516:	learn: 0.9291320	total: 2m 3s	remaining: 1m 55s
517:	learn: 0.9291323	total: 2m 4s	remaining: 1m 55s
518:	learn: 0.9291361	total: 2m 4s	remaining: 1m 55s
519:	learn: 0.9291362	total: 2m 4s	remaining: 1m 55s
52

655:	learn: 0.9313543	total: 2m 33s	remaining: 1m 20s
656:	learn: 0.9313556	total: 2m 33s	remaining: 1m 19s
657:	learn: 0.9313561	total: 2m 33s	remaining: 1m 19s
658:	learn: 0.9313567	total: 2m 33s	remaining: 1m 19s
659:	learn: 0.9313574	total: 2m 33s	remaining: 1m 19s
660:	learn: 0.9317340	total: 2m 33s	remaining: 1m 18s
661:	learn: 0.9317413	total: 2m 34s	remaining: 1m 18s
662:	learn: 0.9317597	total: 2m 34s	remaining: 1m 18s
663:	learn: 0.9317628	total: 2m 34s	remaining: 1m 18s
664:	learn: 0.9317839	total: 2m 34s	remaining: 1m 17s
665:	learn: 0.9317844	total: 2m 34s	remaining: 1m 17s
666:	learn: 0.9317888	total: 2m 34s	remaining: 1m 17s
667:	learn: 0.9319868	total: 2m 35s	remaining: 1m 17s
668:	learn: 0.9320292	total: 2m 35s	remaining: 1m 16s
669:	learn: 0.9320357	total: 2m 35s	remaining: 1m 16s
670:	learn: 0.9320359	total: 2m 35s	remaining: 1m 16s
671:	learn: 0.9321053	total: 2m 35s	remaining: 1m 16s
672:	learn: 0.9321418	total: 2m 35s	remaining: 1m 15s
673:	learn: 0.9321432	total:

811:	learn: 0.9340790	total: 2m 58s	remaining: 41.2s
812:	learn: 0.9340809	total: 2m 58s	remaining: 41s
813:	learn: 0.9341728	total: 2m 58s	remaining: 40.8s
814:	learn: 0.9341729	total: 2m 58s	remaining: 40.5s
815:	learn: 0.9341756	total: 2m 58s	remaining: 40.3s
816:	learn: 0.9341759	total: 2m 58s	remaining: 40.1s
817:	learn: 0.9341929	total: 2m 59s	remaining: 39.9s
818:	learn: 0.9342058	total: 2m 59s	remaining: 39.6s
819:	learn: 0.9342078	total: 2m 59s	remaining: 39.4s
820:	learn: 0.9342138	total: 2m 59s	remaining: 39.2s
821:	learn: 0.9342190	total: 2m 59s	remaining: 38.9s
822:	learn: 0.9342196	total: 3m	remaining: 38.7s
823:	learn: 0.9342198	total: 3m	remaining: 38.5s
824:	learn: 0.9342317	total: 3m	remaining: 38.2s
825:	learn: 0.9342986	total: 3m	remaining: 38s
826:	learn: 0.9343062	total: 3m	remaining: 37.8s
827:	learn: 0.9343098	total: 3m	remaining: 37.6s
828:	learn: 0.9343099	total: 3m	remaining: 37.3s
829:	learn: 0.9343103	total: 3m 1s	remaining: 37.1s
830:	learn: 0.9343180	tota

968:	learn: 0.9358391	total: 3m 23s	remaining: 6.5s
969:	learn: 0.9358392	total: 3m 23s	remaining: 6.28s
970:	learn: 0.9358417	total: 3m 23s	remaining: 6.07s
971:	learn: 0.9358435	total: 3m 23s	remaining: 5.86s
972:	learn: 0.9358961	total: 3m 23s	remaining: 5.65s
973:	learn: 0.9358985	total: 3m 23s	remaining: 5.44s
974:	learn: 0.9358987	total: 3m 24s	remaining: 5.23s
975:	learn: 0.9359344	total: 3m 24s	remaining: 5.02s
976:	learn: 0.9359405	total: 3m 24s	remaining: 4.81s
977:	learn: 0.9359412	total: 3m 24s	remaining: 4.6s
978:	learn: 0.9359419	total: 3m 24s	remaining: 4.39s
979:	learn: 0.9361339	total: 3m 24s	remaining: 4.18s
980:	learn: 0.9361446	total: 3m 25s	remaining: 3.97s
981:	learn: 0.9361452	total: 3m 25s	remaining: 3.76s
982:	learn: 0.9361454	total: 3m 25s	remaining: 3.55s
983:	learn: 0.9361672	total: 3m 25s	remaining: 3.34s
984:	learn: 0.9361719	total: 3m 25s	remaining: 3.13s
985:	learn: 0.9361817	total: 3m 25s	remaining: 2.92s
986:	learn: 0.9361825	total: 3m 26s	remaining: 2

stacking(meta_model=LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False),
     mod=[LGBMRegressor(bagging_fraction=0.85, bagging_freq=1, bagging_seed=23,
       boosting='gbdt', boosting_type='gbdt', class_weight=None,
       colsample_bytree=1.0, feature_fraction=0.8, importance_type='split',
       lambda_l1=0.277, learning_rate=0.01, max_depth=-1, metric='rmse',
       min_...bsample=0.5213,
       verbosity=1), <catboost.core.CatBoostRegressor object at 0x00000197766B17F0>])

In [16]:
pred_stack = stack_model.predict(test.values)

In [17]:
online_score(pred_stack)
#stacking 后的模型相较于LGB单模提升了0.02

预测结果最大值：15525.905392681045,预测结果最小值：1132.9789545971175
对比919分数:0.9851717665913291
